In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.externals.six import StringIO 
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.metrics import make_scorer
import sklearn


# XGBoost
import xgboost as xgb

# Pandas
import pandas as pd

# Numpy
import numpy as np
# Set random seed
np.random.seed(0)

# Plots
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import Image
import pydotplus

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_csv('KeplerData.csv')
target = 'Exoplanet Archive Disposition'
# can't include these columns because they are all null
invalids = ['Equilibrium Temperature Upper Uncertainty [K]', 'Equilibrium Temperature Lower Uncertainty [K]']
# these are all the features
features = list(set(df.columns[4:]) - set(invalids))
data = df[subset_features + [target]]
withoutna = data.dropna()

NameError: name 'subset_features' is not defined

In [ ]:
train, test = sklearn.model_selection.train_test_split(withoutna)
y_train = train[target]
X_train = train[features]
y_test = test[target]
X_test = test[features]

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
confusion = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], normalize=True)
sns.heatmap(confusion, annot=True)

In [ ]:
cv_scores_rf = cross_val_score(clf, X_test, y_test, cv=5)

print("Accuracy:\t\t", np.mean(cv_scores_rf))
print("Completeness:\t\t", sklearn.metrics.recall_score(y_test, y_pred, average='micro'))
print("Contamination:\t\t", 1 - sklearn.metrics.precision_score(y_test, y_pred, average='micro'))

In [ ]:
values = list(zip(subset_features, clf.feature_importances_))
plt.barh(range(len(clf.feature_importances_)), [x[1] for x in values])
sortd = sorted(values, reverse = True, key= lambda x: x[1])[:10]
most_important = [s[0] for s in sortd]
sortd

In [ ]:
# here we look at the most important feature
fig, axs = plt.subplots(ncols=1)
data = df[['Transit Duration Upper Uncertainty [hrs]', target]]
data[data < .5].groupby(target).plot(kind='kde', ax=axs)
data[data < .5].groupby(target).describe()

In [ ]:
# Rerun using only the top 10 most important features
important = withoutna[most_important + [target]]
train, test = sklearn.model_selection.train_test_split(important)
y_train = train[target]
X_train = train[most_important]
y_test = test[target]
X_test = test[most_important]
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
confusion = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], normalize=True)
sns.heatmap(confusion, annot=True)
cv_scores_rf = cross_val_score(clf, X_test, y_test, cv=5)

print("Accuracy:\t\t", np.mean(cv_scores_rf))
print("Completeness:\t\t", sklearn.metrics.recall_score(y_test, y_pred, average='micro'))
print("Contamination:\t\t", 1 - sklearn.metrics.precision_score(y_test, y_pred, average='micro'))